In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('../../dataset/final/imputed_processed_data.csv')
data

,date,price,anuradhapura_producer_price,kurunegala_producer_price,polonnaruwa_producer_price,production,production_total,exchange_rate,fuel_price,m0,m2,m1,m2b
0,1996-01-01,18.07,7.75,7.900,8.18,1333.600,2065.100,53.685,12.4,77071,228090,73783,260481
1,1996-02-01,18.70,8.17,8.260,8.76,1333.600,2065.100,53.685,12.4,79586,234199,77440,269496
2,1996-03-01,18.49,7.95,7.850,7.99,1333.600,2065.100,53.685,12.4,82915,239178,81796,272552
3,1996-04-01,18.52,8.25,8.010,8.29,1333.600,2065.100,54.735,12.4,80873,241893,79478,278425
4,1996-05-01,19.02,8.38,8.500,8.34,731.500,2065.100,54.950,12.4,83410,242072,76620,277365
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2022-08-01,239.24,112.60,113.863,122.93,1461.675,3392.875,357.880,430.0,1386248,10285938,1534114,11986911
320,2022-09-01,228.44,110.50,108.008,117.00,1461.675,3392.875,365.500,430.0,1378579,10351444,1528595,12051480
321,2022-10-01,224.78,105.48,106.740,109.50,1931.200,3392.875,363.000,430.0,1341681,10338384,1456688,12045719
322,2022-11-01,222.28,98.40,101.900,103.48,1931.200,3392.875,368.500,430.0,1318493,10416961,1457977,12162934


In [2]:
# # Standardize time series data
# from pandas import read_csv
# from sklearn.preprocessing import StandardScaler
# from math import sqrt

# # load the dataset and print the first 5 rows
# series = read_csv('daily-minimum-temperatures-in-me.csv', header=0, index_col=0)
# print(series.head())

# # prepare data for standardization
# values = series.values
# values = values.reshape((len(values), 1))
# # train the standardization
# scaler = StandardScaler()
# scaler = scaler.fit(values)
# print('Mean: %f, StandardDeviation: %f' % (scaler.mean_, sqrt(scaler.var_)))
# # standardization the dataset and print the first 5 rows
# normalized = scaler.transform(values)
# for i in range(5):
#  print(normalized[i])
# # inverse transform and print the first 5 rows
# inversed = scaler.inverse_transform(normalized)
# for i in range(5):
#  print(inversed[i])

In [3]:
from temporal_fusion_transformer import TFT

tft_model = TFT(data)

In [4]:
def classify_yaha_mala(row):
    month = row['month']
    
    if month >= 10 or month <= 4:
        return 'maha'
    else:
        return 'yala'

In [5]:
tft_model.preprocess_data(['price', 'exchange_rate'], classify_yaha_mala)

In [6]:
tft_model.create_ts_dataset()
tft_model.create_dataloaders()

In [7]:
tft_model.optimize_hyperparameters(n_trials=2, max_epochs=3, use_learning_rate_finder=False)

[I 2023-04-30 19:29:01,344] A new study created in memory with name: no-name-7c798c30-0c18-47e0-85cc-f86fcd472278
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=3` reached.
[I 2023-04-30 19:29:11,089] Trial 0 finished with value: 555722.625 and parameters: {'gradient_clip_val': 0.046478377430364065, 'hidden_size': 11, 'dropout': 0.17404616821650415, 'hidden_continuous_size': 10, 'attention_head_size': 1, 'learning_rate': 0.0019526740597429282}. Best is trial 0 with value: 555722.625.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=3` reached.
[I 2023-04-30 19:30:06,076] Trial 1 finished with value: 503002.5 and parameters: {'gradient_clip_val': 0.6167000159290529, 'hidden_size': 88, 'dropout': 0.12268131340181626, 'hidden

In [8]:
from pytorch_forecasting.metrics import ImplicitQuantileNetworkDistributionLoss

tft_model.configure_network_and_trainer(loss=ImplicitQuantileNetworkDistributionLoss(), max_epochs=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Number of parameters in network: 580.4k


In [9]:
tft_model.fit_network()


   | Name                               | Type                                    | Params
------------------------------------------------------------------------------------------------
0  | loss                               | ImplicitQuantileNetworkDistributionLoss | 1.9 K 
1  | logging_metrics                    | ModuleList                              | 0     
2  | input_embeddings                   | MultiEmbedding                          | 4     
3  | prescalers                         | ModuleDict                              | 816   
4  | static_variable_selection          | VariableSelectionNetwork                | 46.4 K
5  | encoder_variable_selection         | VariableSelectionNetwork                | 76.8 K
6  | decoder_variable_selection         | VariableSelectionNetwork                | 61.2 K
7  | static_context_variable_selection  | GatedResidualNetwork                    | 31.5 K
8  | static_context_initial_hidden_lstm | GatedResidualNetwork                    |

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [10]:
forecasts, metrics = tft_model.evaluate()
forecast

AttributeError: 'DataLoader' object has no attribute 'target_scale'

In [ ]:
from pytorch_forecasting.data import GroupNormalizer, NaNLabelEncoder

 group_normalizer.inverse_transform(normalized_predictions, val_dataloader.X)

In [ ]:
metrics

In [ ]:
tft_model.plot_forecasts()